<a href="https://colab.research.google.com/github/MBrandao07/Prevencao-a-Fraudes/blob/main/Codigos/2_Preven%C3%A7%C3%A3o_a_Fraude_Data_Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparação dos dados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# conectando ao drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Funções

In [ ]:
def generate_metadata(dataframe):
    """
    Gera um dataframe contendo metadados das colunas do dataframe fornecido.

    :param dataframe: DataFrame para o qual os metadados serão gerados.
    :return: DataFrame contendo metadados.
    """

    # Coleta de metadados básicos
    metadata = pd.DataFrame({
        'nome_variavel': dataframe.columns,
        'tipo': dataframe.dtypes,
        'qt_nulos': dataframe.isnull().sum(),
        'percent_nulos': round((dataframe.isnull().sum() / len(dataframe))* 100,2),
        'cardinalidade': dataframe.nunique(),
    })
    metadata=metadata.sort_values(by='percent_nulos',ascending=False)
    metadata = metadata.reset_index(drop=True)

    return metadata

### Importando as bases de dados

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/2 - Projetos Portfolio/Prevencao de Fraude/Base de dados/train.csv")
train.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD
0,59383,2021-08-01 00:04:37,323,217,4.60,0
1,59384,2021-08-01 00:12:10,6,429,8.61,0
2,59385,2021-08-01 00:12:34,714,1011,64.00,0
3,59386,2021-08-01 00:15:40,266,1969,12.72,0
4,59387,2021-08-01 00:16:01,890,1482,98.88,0


In [ ]:
train.shape

(291231, 6)

In [ ]:
test = pd.read_csv("/content/drive/MyDrive/2 - Projetos Portfolio/Prevencao de Fraude/Base de dados/test.csv")
test.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
0,352590,2022-01-01 00:03:14,208,640,147.24
1,352591,2022-01-01 00:26:39,851,315,13.27
2,352592,2022-01-01 00:30:30,454,27,70.52
3,352593,2022-01-01 00:31:31,677,1573,44.75
4,352594,2022-01-01 00:56:18,322,1398,37.60


In [ ]:
test.shape

(226731, 5)

In [ ]:
customer = pd.read_csv("/content/drive/MyDrive/2 - Projetos Portfolio/Prevencao de Fraude/Base de dados/customer.csv")
customer.head()

,CUSTOMER_ID,x_customer_id,y_customer_id,mean_amount,std_amount,mean_nb_tx_per_day
0,0,10.950170,59.768684,62.262521,31.131260,2.179533
1,1,13.671851,52.775318,46.570785,23.285393,3.567092
2,2,-9.381829,38.617619,80.213879,40.106939,2.115580
3,3,-3.745116,40.551744,11.748426,5.874213,0.348517
4,4,4.312412,51.067100,78.924891,39.462446,3.480049


In [ ]:
customer.shape

(1000, 6)

In [ ]:
terminal = pd.read_csv("/content/drive/MyDrive/2 - Projetos Portfolio/Prevencao de Fraude/Base de dados/terminal.csv")
terminal.head()

,TERMINAL_ID,x_terminal_id,y_terminal_id
0,0,9.388886,44.298820
1,1,12.204779,38.423219
2,2,16.123444,41.642938
3,3,13.341542,37.858452
4,4,11.964165,55.351051


In [ ]:
terminal.shape

(2000, 3)

### Mesclando as bases

In [ ]:
# base treino
df_train = pd.merge(train, customer, on='CUSTOMER_ID', how='left')
df_train_merged = pd.merge(df_train, terminal, on='TERMINAL_ID', how='left')
df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,mean_nb_tx_per_day,x_terminal_id,y_terminal_id
0,59383,2021-08-01 00:04:37,323,217,4.60,0,-0.362432,39.512573,7.353061,3.676530,3.324124,2.439994,50.324767
1,59384,2021-08-01 00:12:10,6,429,8.61,0,-0.466155,38.210214,18.618562,9.309281,3.778676,11.450431,49.902275
2,59385,2021-08-01 00:12:34,714,1011,64.00,0,-0.682848,38.095822,82.620413,41.310207,3.723765,13.616103,46.150744
3,59386,2021-08-01 00:15:40,266,1969,12.72,0,14.309093,40.607318,9.852171,4.926085,3.862067,12.286148,45.514582
4,59387,2021-08-01 00:16:01,890,1482,98.88,0,-4.209270,36.454733,83.660035,41.830018,3.128315,5.168861,51.234704


In [ ]:
df_train_merged.shape

(291231, 13)

In [ ]:
# base test
df_test = pd.merge(test, customer, on='CUSTOMER_ID', how='left')
df_test_merged = pd.merge(df_test, terminal, on='TERMINAL_ID', how='left')
df_test_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,x_customer_id,y_customer_id,mean_amount,std_amount,mean_nb_tx_per_day,x_terminal_id,y_terminal_id
0,352590,2022-01-01 00:03:14,208,640,147.24,5.008306,52.371948,82.134163,41.067082,1.917538,-9.006138,38.793968
1,352591,2022-01-01 00:26:39,851,315,13.27,11.299561,43.927394,46.525169,23.262584,1.675154,0.106110,38.908842
2,352592,2022-01-01 00:30:30,454,27,70.52,18.079634,59.234900,62.594881,31.297441,3.956352,11.748156,50.205347
3,352593,2022-01-01 00:31:31,677,1573,44.75,-15.536839,39.117718,56.856095,28.428048,3.756383,12.594341,51.499157
4,352594,2022-01-01 00:56:18,322,1398,37.60,8.942504,45.737756,35.430997,17.715499,3.787229,14.352046,52.014614


In [ ]:
df_test_merged.shape

(226731, 12)

### Convertendo as colunas TX_DATETIME para o formato Datetime

In [ ]:
# Convertendo a coluna TX_DATETIME para datetime
df_train_merged['TX_DATETIME'] = pd.to_datetime(df_train_merged['TX_DATETIME'])
df_test_merged['TX_DATETIME'] = pd.to_datetime(df_test_merged['TX_DATETIME'])

# Criando a variável que indica o mês de referência
df_train_merged['month_reference'] = df_train_merged['TX_DATETIME'].dt.strftime('%Y%m')
df_test_merged['month_reference'] = df_test_merged['TX_DATETIME'].dt.strftime('%Y%m')

# Feature Engineering

## Criando novas variáveis

### Periodo do dia

In [ ]:
# Função para classificar o período do dia
def classify_period(hour):
    if 6 <= hour < 12:
        return 'manhã'
    elif 12 <= hour < 18:
        return 'tarde'
    else:
        return 'noite'

# Aplicar a função para criar a nova coluna 'PERIODO_DIA'
df_train_merged['PERIODO_DIA'] = df_train_merged['TX_DATETIME'].dt.hour.apply(classify_period)
df_test_merged['PERIODO_DIA'] = df_test_merged['TX_DATETIME'].dt.hour.apply(classify_period)

### Variáveis "hora do dia", "dia da semana", "dia do mês", "mês", "ano" e "flag fim de semana"

In [ ]:
# Extração de features temporais para o DataFrame de treino
df_train_merged['hour_of_day'] = df_train_merged['TX_DATETIME'].dt.hour          # Hora do dia
df_train_merged['day_of_week'] = df_train_merged['TX_DATETIME'].dt.dayofweek     # Dia da semana (0=segunda-feira, 6=domingo)
df_train_merged['day_of_month'] = df_train_merged['TX_DATETIME'].dt.day          # Dia do mês
df_train_merged['month'] = df_train_merged['TX_DATETIME'].dt.month               # Mês
df_train_merged['year'] = df_train_merged['TX_DATETIME'].dt.year                 # Ano
df_train_merged['is_weekend'] = df_train_merged['TX_DATETIME'].dt.dayofweek >= 5 # Flag para finais de semana

# # Extração de features temporais para o DataFrame de teste
df_test_merged['hour_of_day'] = df_test_merged['TX_DATETIME'].dt.hour          # Hora do dia
df_test_merged['day_of_week'] = df_test_merged['TX_DATETIME'].dt.dayofweek     # Dia da semana (0=segunda-feira, 6=domingo)
df_test_merged['day_of_month'] = df_test_merged['TX_DATETIME'].dt.day          # Dia do mês
df_test_merged['month'] = df_test_merged['TX_DATETIME'].dt.month               # Mês
df_test_merged['year'] = df_test_merged['TX_DATETIME'].dt.year                 # Ano
df_test_merged['is_weekend'] = df_test_merged['TX_DATETIME'].dt.dayofweek >= 5 # Flag para finais de semana

### Diferença de tempo entre duas transações

In [ ]:
# Ordenar por CUSTOMER_ID e TX_DATETIME
df_train_merged = df_train_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])

# Calcular a diferença de tempo entre transações consecutivas do mesmo cliente
df_train_merged['time_diff_transactions'] = df_train_merged.groupby('CUSTOMER_ID')['TX_DATETIME'].diff()

# Ordenar por CUSTOMER_ID e TX_DATETIME
df_test_merged = df_test_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])

# Calcular a diferença de tempo entre transações consecutivas do mesmo cliente
df_test_merged['time_diff_transactions'] = df_test_merged.groupby('CUSTOMER_ID')['TX_DATETIME'].diff()

df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,y_terminal_id,month_reference,PERIODO_DIA,hour_of_day,day_of_week,day_of_month,month,year,is_weekend,time_diff_transactions
69,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,59.061828,202108,noite,3,6,1,8,2021,True,NaT
209,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,37.452541,202108,noite,5,6,1,8,2021,True,0 days 02:29:38
749,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,57.292877,202108,manhã,10,6,1,8,2021,True,0 days 05:09:34
1338,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,61.073466,202108,tarde,15,6,1,8,2021,True,0 days 04:58:13
1650,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,39.053767,202108,noite,19,6,1,8,2021,True,0 days 03:47:05


### Distância do cliente ao terminal

In [ ]:
import math

def haversine(lon1, lat1, lon2, lat2):
    # Converter de graus para radianos
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Diferenças das coordenadas
    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad

    # Fórmula de Haversine
    a = math.sin(delta_lat / 2) ** 2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(delta_lon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Raio da Terra em km
    R = 6371.0

    # Distância
    distancia = R * c
    return distancia

In [ ]:
df_train_merged['distance_customer_terminal'] = df_train_merged.apply(
    lambda row: haversine(row['x_customer_id'], row['y_customer_id'], row['x_terminal_id'], row['y_terminal_id']),
    axis=1
)

df_test_merged['distance_customer_terminal'] = df_test_merged.apply(
    lambda row: haversine(row['x_customer_id'], row['y_customer_id'], row['x_terminal_id'], row['y_terminal_id']),
    axis=1
)

In [ ]:
df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,month_reference,PERIODO_DIA,hour_of_day,day_of_week,day_of_month,month,year,is_weekend,time_diff_transactions,distance_customer_terminal
69,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,202108,noite,3,6,1,8,2021,True,NaT,157.692238
209,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,202108,noite,5,6,1,8,2021,True,0 days 02:29:38,2762.732805
749,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,202108,manhã,10,6,1,8,2021,True,0 days 05:09:34,920.672509
1338,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,202108,tarde,15,6,1,8,2021,True,0 days 04:58:13,145.190544
1650,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,202108,noite,19,6,1,8,2021,True,0 days 03:47:05,2593.697073


### Valor movimentado nas últimas X horas

In [ ]:
# Exemplo de valores de janela de tempo
time_windows = ['1H', '2H', '4H', '8H', '12H', '24H', '48H', '72H', '7D', '14D', '21D', '30D', '45D']

# Ordena o DataFrame por 'CUSTOMER_ID' e 'TX_DATETIME'
df_train_merged = df_train_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])
df_test_merged = df_test_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])


# Loop para aplicar a função de soma de valores de transações em diferentes janelas de tempo
for time_window in time_windows:
    # Nome da coluna baseada na janela de tempo
    column_name = f'TOTAL_AMOUNT_PER_CUSTOMER_LAST_{time_window.upper()}'

    # Função para calcular a soma dos valores das transações nas últimas 'time_window' para cada cliente
    def calculate_total_amount_recent(transactions):
        return transactions.rolling(time_window, on='TX_DATETIME')['TX_AMOUNT'].sum()

    # Aplica a função no contexto do groupby para cada CUSTOMER_ID e adiciona ao dataframe
    df_train_merged[column_name] = df_train_merged.groupby('CUSTOMER_ID').apply(calculate_total_amount_recent).reset_index(level=0, drop=True)
    df_test_merged[column_name] = df_test_merged.groupby('CUSTOMER_ID').apply(calculate_total_amount_recent).reset_index(level=0, drop=True)

# Exibe as primeiras linhas do dataframe resultante
df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,TOTAL_AMOUNT_PER_CUSTOMER_LAST_8H,TOTAL_AMOUNT_PER_CUSTOMER_LAST_12H,TOTAL_AMOUNT_PER_CUSTOMER_LAST_24H,TOTAL_AMOUNT_PER_CUSTOMER_LAST_48H,TOTAL_AMOUNT_PER_CUSTOMER_LAST_72H,TOTAL_AMOUNT_PER_CUSTOMER_LAST_7D,TOTAL_AMOUNT_PER_CUSTOMER_LAST_14D,TOTAL_AMOUNT_PER_CUSTOMER_LAST_21D,TOTAL_AMOUNT_PER_CUSTOMER_LAST_30D,TOTAL_AMOUNT_PER_CUSTOMER_LAST_45D
69,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51
209,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,191.12,191.12,191.12,191.12,191.12,191.12,191.12,191.12,191.12,191.12
749,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,287.62,287.62,287.62,287.62,287.62,287.62,287.62,287.62,287.62,287.62
1338,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,178.69,308.30,369.81,369.81,369.81,369.81,369.81,369.81,369.81,369.81
1650,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,132.96,229.46,420.58,420.58,420.58,420.58,420.58,420.58,420.58,420.58


### Média do valor movimentado nas últimas X horas

In [ ]:
# Exemplo de valores de janela de tempo
time_windows = ['1H', '2H', '4H', '8H', '12H', '24H', '48H', '72H', '7D', '14D', '21D', '30D', '45D']

# Ordena o DataFrame por 'CUSTOMER_ID' e 'TX_DATETIME'
df_train_merged = df_train_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])
df_test_merged = df_test_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])

# Loop para aplicar a função de soma de valores de transações em diferentes janelas de tempo
for time_window in time_windows:
    # Nome da coluna baseada na janela de tempo
    column_name = f'MEAN_AMOUNT_PER_CUSTOMER_LAST_{time_window.upper()}'

    # Função para calcular a soma dos valores das transações nas últimas 'time_window' para cada cliente
    def calculate_mean_amount_recent(transactions):
        return transactions.rolling(time_window, on='TX_DATETIME')['TX_AMOUNT'].mean()

    # Aplica a função no contexto do groupby para cada CUSTOMER_ID e adiciona ao dataframe
    df_train_merged[column_name] = df_train_merged.groupby('CUSTOMER_ID').apply(calculate_mean_amount_recent).reset_index(level=0, drop=True)
    df_test_merged[column_name] = df_test_merged.groupby('CUSTOMER_ID').apply(calculate_mean_amount_recent).reset_index(level=0, drop=True)

# Exibe as primeiras linhas do dataframe resultante
df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,MEAN_AMOUNT_PER_CUSTOMER_LAST_8H,MEAN_AMOUNT_PER_CUSTOMER_LAST_12H,MEAN_AMOUNT_PER_CUSTOMER_LAST_24H,MEAN_AMOUNT_PER_CUSTOMER_LAST_48H,MEAN_AMOUNT_PER_CUSTOMER_LAST_72H,MEAN_AMOUNT_PER_CUSTOMER_LAST_7D,MEAN_AMOUNT_PER_CUSTOMER_LAST_14D,MEAN_AMOUNT_PER_CUSTOMER_LAST_21D,MEAN_AMOUNT_PER_CUSTOMER_LAST_30D,MEAN_AMOUNT_PER_CUSTOMER_LAST_45D
69,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,61.510000,61.510000,61.510000,61.510000,61.510000,61.510000,61.510000,61.510000,61.510000,61.510000
209,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,95.560000,95.560000,95.560000,95.560000,95.560000,95.560000,95.560000,95.560000,95.560000,95.560000
749,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,95.873333,95.873333,95.873333,95.873333,95.873333,95.873333,95.873333,95.873333,95.873333,95.873333
1338,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,89.345000,102.766667,92.452500,92.452500,92.452500,92.452500,92.452500,92.452500,92.452500,92.452500
1650,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,66.480000,76.486667,84.116000,84.116000,84.116000,84.116000,84.116000,84.116000,84.116000,84.116000


### Mediana do valor movimentado nas últimas X horas

In [ ]:
# Exemplo de valores de janela de tempo
time_windows = ['1H', '2H', '4H', '8H', '12H', '24H', '48H', '72H', '7D', '14D', '21D', '30D', '45D']

# Ordena o DataFrame por 'CUSTOMER_ID' e 'TX_DATETIME'
df_train_merged = df_train_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])
df_test_merged = df_test_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])

# Loop para aplicar a função de soma de valores de transações em diferentes janelas de tempo
for time_window in time_windows:
    # Nome da coluna baseada na janela de tempo
    column_name = f'MEDIAN_AMOUNT_PER_CUSTOMER_LAST_{time_window.upper()}'

    # Função para calcular a soma dos valores das transações nas últimas 'time_window' para cada cliente
    def calculate_median_amount_recent(transactions):
        return transactions.rolling(time_window, on='TX_DATETIME')['TX_AMOUNT'].median()

    # Aplica a função no contexto do groupby para cada CUSTOMER_ID e adiciona ao dataframe
    df_train_merged[column_name] = df_train_merged.groupby('CUSTOMER_ID').apply(calculate_median_amount_recent).reset_index(level=0, drop=True)
    df_test_merged[column_name] = df_test_merged.groupby('CUSTOMER_ID').apply(calculate_median_amount_recent).reset_index(level=0, drop=True)

# Exibe as primeiras linhas do dataframe resultante
df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,MEDIAN_AMOUNT_PER_CUSTOMER_LAST_8H,MEDIAN_AMOUNT_PER_CUSTOMER_LAST_12H,MEDIAN_AMOUNT_PER_CUSTOMER_LAST_24H,MEDIAN_AMOUNT_PER_CUSTOMER_LAST_48H,MEDIAN_AMOUNT_PER_CUSTOMER_LAST_72H,MEDIAN_AMOUNT_PER_CUSTOMER_LAST_7D,MEDIAN_AMOUNT_PER_CUSTOMER_LAST_14D,MEDIAN_AMOUNT_PER_CUSTOMER_LAST_21D,MEDIAN_AMOUNT_PER_CUSTOMER_LAST_30D,MEDIAN_AMOUNT_PER_CUSTOMER_LAST_45D
69,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,61.510,61.51,61.510,61.510,61.510,61.510,61.510,61.510,61.510,61.510
209,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,95.560,95.56,95.560,95.560,95.560,95.560,95.560,95.560,95.560,95.560
749,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,96.500,96.50,96.500,96.500,96.500,96.500,96.500,96.500,96.500,96.500
1338,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,89.345,96.50,89.345,89.345,89.345,89.345,89.345,89.345,89.345,89.345
1650,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,66.480,82.19,82.190,82.190,82.190,82.190,82.190,82.190,82.190,82.190


### Mínimo valor movimentado nas últimas X horas

In [ ]:
# Exemplo de valores de janela de tempo
time_windows = ['1H', '2H', '4H', '8H', '12H', '24H', '48H', '72H', '7D', '14D', '21D', '30D', '45D']

# Ordena o DataFrame por 'CUSTOMER_ID' e 'TX_DATETIME'
df_train_merged = df_train_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])
df_test_merged = df_test_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])

# Loop para aplicar a função de soma de valores de transações em diferentes janelas de tempo
for time_window in time_windows:
    # Nome da coluna baseada na janela de tempo
    column_name = f'MIN_AMOUNT_PER_CUSTOMER_LAST_{time_window.upper()}'

    # Função para calcular a soma dos valores das transações nas últimas 'time_window' para cada cliente
    def calculate_min_amount_recent(transactions):
        return transactions.rolling(time_window, on='TX_DATETIME')['TX_AMOUNT'].min()

    # Aplica a função no contexto do groupby para cada CUSTOMER_ID e adiciona ao dataframe
    df_train_merged[column_name] = df_train_merged.groupby('CUSTOMER_ID').apply(calculate_min_amount_recent).reset_index(level=0, drop=True)
    df_test_merged[column_name] = df_test_merged.groupby('CUSTOMER_ID').apply(calculate_min_amount_recent).reset_index(level=0, drop=True)

# Exibe as primeiras linhas do dataframe resultante
df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,MIN_AMOUNT_PER_CUSTOMER_LAST_8H,MIN_AMOUNT_PER_CUSTOMER_LAST_12H,MIN_AMOUNT_PER_CUSTOMER_LAST_24H,MIN_AMOUNT_PER_CUSTOMER_LAST_48H,MIN_AMOUNT_PER_CUSTOMER_LAST_72H,MIN_AMOUNT_PER_CUSTOMER_LAST_7D,MIN_AMOUNT_PER_CUSTOMER_LAST_14D,MIN_AMOUNT_PER_CUSTOMER_LAST_21D,MIN_AMOUNT_PER_CUSTOMER_LAST_30D,MIN_AMOUNT_PER_CUSTOMER_LAST_45D
69,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51
209,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51
749,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51
1338,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,82.19,82.19,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51
1650,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,50.77,50.77,50.77,50.77,50.77,50.77,50.77,50.77,50.77,50.77


### Máximo valor movimentado nas últimas x Horas

In [ ]:
# Exemplo de valores de janela de tempo
time_windows = ['1H', '2H', '4H', '8H', '12H', '24H', '48H', '72H', '7D', '14D', '21D', '30D', '45D']

# Ordena o DataFrame por 'CUSTOMER_ID' e 'TX_DATETIME'
df_train_merged = df_train_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])
df_test_merged = df_test_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])

# Loop para aplicar a função de soma de valores de transações em diferentes janelas de tempo
for time_window in time_windows:
    # Nome da coluna baseada na janela de tempo
    column_name = f'MAX_AMOUNT_PER_CUSTOMER_LAST_{time_window.upper()}'

    # Função para calcular a soma dos valores das transações nas últimas 'time_window' para cada cliente
    def calculate_max_amount_recent(transactions):
        return transactions.rolling(time_window, on='TX_DATETIME')['TX_AMOUNT'].max()

    # Aplica a função no contexto do groupby para cada CUSTOMER_ID e adiciona ao dataframe
    df_train_merged[column_name] = df_train_merged.groupby('CUSTOMER_ID').apply(calculate_max_amount_recent).reset_index(level=0, drop=True)
    df_test_merged[column_name] = df_test_merged.groupby('CUSTOMER_ID').apply(calculate_max_amount_recent).reset_index(level=0, drop=True)

# Exibe as primeiras linhas do dataframe resultante
df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,MAX_AMOUNT_PER_CUSTOMER_LAST_8H,MAX_AMOUNT_PER_CUSTOMER_LAST_12H,MAX_AMOUNT_PER_CUSTOMER_LAST_24H,MAX_AMOUNT_PER_CUSTOMER_LAST_48H,MAX_AMOUNT_PER_CUSTOMER_LAST_72H,MAX_AMOUNT_PER_CUSTOMER_LAST_7D,MAX_AMOUNT_PER_CUSTOMER_LAST_14D,MAX_AMOUNT_PER_CUSTOMER_LAST_21D,MAX_AMOUNT_PER_CUSTOMER_LAST_30D,MAX_AMOUNT_PER_CUSTOMER_LAST_45D
69,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51,61.51
209,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,129.61,129.61,129.61,129.61,129.61,129.61,129.61,129.61,129.61,129.61
749,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,129.61,129.61,129.61,129.61,129.61,129.61,129.61,129.61,129.61,129.61
1338,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,96.50,129.61,129.61,129.61,129.61,129.61,129.61,129.61,129.61,129.61
1650,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,82.19,96.50,129.61,129.61,129.61,129.61,129.61,129.61,129.61,129.61


### Quantidade total de transações realizadas pelo cliente nas últimas X horas

In [ ]:
# Exemplo de valores de janela de tempo
time_windows = ['1H', '2H', '4H', '8H', '12H', '24H', '48H', '72H', '7D', '14D', '21D', '30D', '45D']

# Loop para aplicar a função de contagem de transações em diferentes janelas de tempo
for time_window in time_windows:
    # Nome da coluna baseada na janela de tempo
    column_name = f'TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_{time_window.upper()}'

    # Função para calcular a quantidade de transações nas últimas 'time_window' horas para cada cliente
    def calculate_transaction_count_recent(transactions):
        # Contando apenas as transações em uma janela de 'time_window'
        return transactions.rolling(time_window, on='TX_DATETIME')['TX_AMOUNT'].count()

    # Aplica a função no contexto do groupby para cada CUSTOMER_ID e adiciona ao dataframe
    df_train_merged[column_name] = df_train_merged.groupby('CUSTOMER_ID').apply(calculate_transaction_count_recent).reset_index(level=0, drop=True)
    df_test_merged[column_name] = df_test_merged.groupby('CUSTOMER_ID').apply(calculate_transaction_count_recent).reset_index(level=0, drop=True)

# Exibir o dataframe resultante
df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_8H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_12H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_24H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_48H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_72H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_7D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_14D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_21D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_30D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_45D
69,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
209,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
749,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
1338,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,2.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
1650,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,2.0,3.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0


### Tempo desde a última transação

In [ ]:
# Calcula o tempo desde a última transação
df_train_merged['days_since_last_transaction'] = df_train_merged.groupby('CUSTOMER_ID')['TX_DATETIME'].diff().dt.days
df_test_merged['days_since_last_transaction'] = df_test_merged.groupby('CUSTOMER_ID')['TX_DATETIME'].diff().dt.days

df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_12H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_24H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_48H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_72H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_7D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_14D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_21D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_30D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_45D,days_since_last_transaction
69,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
209,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0
749,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0
1338,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,0.0
1650,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,3.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,0.0


### Flag para transações consecutivas no mesmo terminal

In [ ]:
# Flag para transações consecutivas no mesmo terminal
df_train_merged['consecutive_transactions_same_terminal'] = (df_train_merged['TERMINAL_ID'] == df_train_merged.groupby('CUSTOMER_ID')['TERMINAL_ID'].shift(1)).astype(int)
df_test_merged['consecutive_transactions_same_terminal'] = (df_test_merged['TERMINAL_ID'] == df_test_merged.groupby('CUSTOMER_ID')['TERMINAL_ID'].shift(1)).astype(int)

df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_24H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_48H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_72H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_7D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_14D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_21D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_30D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_45D,days_since_last_transaction,consecutive_transactions_same_terminal
69,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,0
209,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0
749,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,0
1338,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,0.0,0
1650,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,0.0,0


### Proporção de transações em períodos de alto risco

In [ ]:
# Define um período de risco, como "noite"
high_risk_periods = ['noite']

# Ordena os DataFrames por 'CUSTOMER_ID' e 'TX_DATETIME'
df_train_merged = df_train_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])
df_test_merged = df_test_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])

# Função para calcular a proporção acumulada de transações em períodos de alto risco
def calculate_high_risk_ratio(transactions):
    # Converte os valores para 1 se estiverem no período de risco, caso contrário, 0
    binary_risk = transactions.isin(high_risk_periods).astype(int)
    # Calcula a média acumulativa
    return binary_risk.expanding().mean()

# Aplica a função de proporção acumulada no contexto do groupby para cada CUSTOMER_ID
df_train_merged['transaction_ratio_high_risk_period'] = df_train_merged.groupby('CUSTOMER_ID')['PERIODO_DIA'].apply(calculate_high_risk_ratio).reset_index(level=0, drop=True)
df_test_merged['transaction_ratio_high_risk_period'] = df_test_merged.groupby('CUSTOMER_ID')['PERIODO_DIA'].apply(calculate_high_risk_ratio).reset_index(level=0, drop=True)

# Exibir o DataFrame resultante
df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_48H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_72H,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_7D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_14D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_21D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_30D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_45D,days_since_last_transaction,consecutive_transactions_same_terminal,transaction_ratio_high_risk_period
69,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,0,1.000000
209,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0,1.000000
749,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,0,0.666667
1338,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,0.0,0,0.500000
1650,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,0.0,0,0.600000


### Flag de transação acima da média do cliente

In [ ]:
# Ordena os DataFrames por 'CUSTOMER_ID' e 'TX_DATETIME'
df_train_merged = df_train_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])
df_test_merged = df_test_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])

# Função para calcular a média cumulativa de transações até o momento da transação atual
def calculate_cumulative_mean(transactions):
    return transactions.expanding().mean()

# Aplica a função para calcular a média cumulativa por cliente no conjunto de treino
df_train_merged['cumulative_avg_amount'] = df_train_merged.groupby('CUSTOMER_ID')['TX_AMOUNT'].apply(calculate_cumulative_mean).reset_index(level=0, drop=True)

# Aplica a função para calcular a média cumulativa por cliente no conjunto de teste
df_test_merged['cumulative_avg_amount'] = df_test_merged.groupby('CUSTOMER_ID')['TX_AMOUNT'].apply(calculate_cumulative_mean).reset_index(level=0, drop=True)

# Define o alerta comparando o valor da transação com a média cumulativa
df_train_merged['high_value_alert'] = (df_train_merged['TX_AMOUNT'] > df_train_merged['cumulative_avg_amount']).astype(int)
df_test_merged['high_value_alert'] = (df_test_merged['TX_AMOUNT'] > df_test_merged['cumulative_avg_amount']).astype(int)

# Visualizar o resultado
df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_7D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_14D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_21D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_30D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_45D,days_since_last_transaction,consecutive_transactions_same_terminal,transaction_ratio_high_risk_period,cumulative_avg_amount,high_value_alert
69,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,1.0,1.0,1.0,1.0,1.0,NaN,0,1.000000,61.510000,0
209,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,2.0,2.0,2.0,2.0,2.0,0.0,0,1.000000,95.560000,1
749,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,3.0,3.0,3.0,3.0,3.0,0.0,0,0.666667,95.873333,1
1338,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,4.0,4.0,4.0,4.0,4.0,0.0,0,0.500000,92.452500,0
1650,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,5.0,5.0,5.0,5.0,5.0,0.0,0,0.600000,84.116000,0


### Contagem de quantidade de terminais distintos utilizados por cliente

In [ ]:
import pandas as pd

# Ordena os DataFrames por 'CUSTOMER_ID' e 'TX_DATETIME'
df_train_merged = df_train_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])
df_test_merged = df_test_merged.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])

# Função para calcular o número total de terminais distintos usados até a transação atual
def calculate_unique_terminals_count(transactions):
    unique_terminals = set()
    unique_terminal_counts = []

    for terminal in transactions:
        unique_terminals.add(terminal)  # Adiciona o terminal ao conjunto de únicos
        unique_terminal_counts.append(len(unique_terminals))  # Adiciona a contagem total de terminais únicos

    return pd.Series(unique_terminal_counts, index=transactions.index)

# Aplica a função para calcular o número total de terminais distintos usados até cada transação no conjunto de treino
df_train_merged['distinct_terminals_count'] = df_train_merged.groupby('CUSTOMER_ID')['TERMINAL_ID'].apply(calculate_unique_terminals_count).reset_index(level=0, drop=True)

# Aplica a função para calcular o número total de terminais distintos usados até cada transação no conjunto de teste
df_test_merged['distinct_terminals_count'] = df_test_merged.groupby('CUSTOMER_ID')['TERMINAL_ID'].apply(calculate_unique_terminals_count).reset_index(level=0, drop=True)

# Visualizar o resultado
df_train_merged.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_FRAUD,x_customer_id,y_customer_id,mean_amount,std_amount,...,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_14D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_21D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_30D,TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_45D,days_since_last_transaction,consecutive_transactions_same_terminal,transaction_ratio_high_risk_period,cumulative_avg_amount,high_value_alert,distinct_terminals_count
69,59452,2021-08-01 03:01:00,0,1133,61.51,0,10.95017,59.768684,62.262521,31.13126,...,1.0,1.0,1.0,1.0,NaN,0,1.000000,61.510000,0,1
209,59592,2021-08-01 05:30:38,0,1138,129.61,0,10.95017,59.768684,62.262521,31.13126,...,2.0,2.0,2.0,2.0,0.0,0,1.000000,95.560000,1,2
749,60132,2021-08-01 10:40:12,0,1530,96.50,0,10.95017,59.768684,62.262521,31.13126,...,3.0,3.0,3.0,3.0,0.0,0,0.666667,95.873333,1,3
1338,60721,2021-08-01 15:38:25,0,241,82.19,0,10.95017,59.768684,62.262521,31.13126,...,4.0,4.0,4.0,4.0,0.0,0,0.500000,92.452500,0,4
1650,61033,2021-08-01 19:25:30,0,1536,50.77,0,10.95017,59.768684,62.262521,31.13126,...,5.0,5.0,5.0,5.0,0.0,0,0.600000,84.116000,0,5


### Razões entre as features

In [ ]:
# Razão entre o valor médio transacionado em 1H pelo valor médio transacionado em XH ou X Dias
df_train_merged['RATIO_MEAN_AMOUNT_PER_CUSTOMER_LAST_1H_TO_2H'] = df_train_merged['MEAN_AMOUNT_PER_CUSTOMER_LAST_1H'] / df_train_merged['MEAN_AMOUNT_PER_CUSTOMER_LAST_2H']
df_train_merged['RATIO_MEAN_AMOUNT_PER_CUSTOMER_LAST_1H_TO_4H'] = df_train_merged['MEAN_AMOUNT_PER_CUSTOMER_LAST_1H'] / df_train_merged['MEAN_AMOUNT_PER_CUSTOMER_LAST_4H']
df_train_merged['RATIO_MEAN_AMOUNT_PER_CUSTOMER_LAST_1H_TO_8H'] = df_train_merged['MEAN_AMOUNT_PER_CUSTOMER_LAST_1H'] / df_train_merged['MEAN_AMOUNT_PER_CUSTOMER_LAST_8H']
df_train_merged['RATIO_MEAN_AMOUNT_PER_CUSTOMER_LAST_1H_TO_24H'] = df_train_merged['MEAN_AMOUNT_PER_CUSTOMER_LAST_1H'] / df_train_merged['MEAN_AMOUNT_PER_CUSTOMER_LAST_24H']
df_train_merged['RATIO_MEAN_AMOUNT_PER_CUSTOMER_LAST_1H_TO_7D'] = df_train_merged['MEAN_AMOUNT_PER_CUSTOMER_LAST_1H'] / df_train_merged['MEAN_AMOUNT_PER_CUSTOMER_LAST_7D']
df_train_merged['RATIO_MEAN_AMOUNT_PER_CUSTOMER_LAST_1H_TO_14D'] = df_train_merged['MEAN_AMOUNT_PER_CUSTOMER_LAST_1H'] / df_train_merged['MEAN_AMOUNT_PER_CUSTOMER_LAST_14D']

# Razão entre o valor médio transacionado em 1H pelo valor médio transacionado em XH ou X Dias
df_train_merged['RATIO_TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_1H_TO_2H'] = df_train_merged['TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_1H'] / df_train_merged['TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_2H']
df_train_merged['RATIO_TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_1H_TO_4H'] = df_train_merged['TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_1H'] / df_train_merged['TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_4H']
df_train_merged['RATIO_TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_1H_TO_8H'] = df_train_merged['TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_1H'] / df_train_merged['TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_8H']
df_train_merged['RATIO_TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_1H_TO_24H'] = df_train_merged['TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_1H'] / df_train_merged['TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_24H']
df_train_merged['RATIO_TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_1H_TO_7D'] = df_train_merged['TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_1H'] / df_train_merged['TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_7D']
df_train_merged['RATIO_TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_1H_TO_14D'] = df_train_merged['TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_1H'] / df_train_merged['TOTAL_TRANSACTIONS_PER_CUSTOMER_LAST_14D']

### Salvando a base tratada em um arquivo CSV

In [ ]:
# base treino
df_train_merged.to_csv('/content/drive/MyDrive/2 - Projetos Portfolio/Prevencao de Fraude/Base de dados/df_train_full.csv', index=False)

# base teste
df_test_merged.to_csv('/content/drive/MyDrive/2 - Projetos Portfolio/Prevencao de Fraude/Base de dados/df_test_full.csv', index=False)